In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import collections
import pickle
import gzip

from tf.app import use

In [3]:
A = use('dss:clone', checkout='clone', hoist=globals())

Using TF-app in /Users/dirk/github/annotation/app-dss/code:
	repo clone offline under ~/github (local github)
Using data in /Users/dirk/github/etcbc/dss/tf/0.3:
	repo clone offline under ~/github (local github)


# Parallels

We make edges between similar lines.

When are lines similar?

If a certain distance metric is above a certain threshold.

We choose this metric:

* we reduce a line to the set of lexemes in it.
* the similarity between two lines is the length of the intersection divided by the length of the union of their sets times 100.

# Preparation

We pre-compute all sets for all lines.

In [4]:
def makeSet(l):
  lineSet = set()
  for s in L.d(l, otype='word'):
    r = F.lex.v(s)
    if r:
      lineSet.add(r)
  return lineSet

In [5]:
lines = {}

for l in F.otype.s('line'):
  lineSet = makeSet(l)
  if lineSet:
    lines[l] = lineSet
    
nLines = len(lines)
print(f'{nLines} ayas')

52760 ayas


# Measure

In [6]:
def sim(lSet, mSet):
  return int(round(100 * len(lSet & mSet) / len(lSet | mSet)))

# Compute all similarities

We are going to perform more than a billion of comparisons, each of which is more than an elemetary operation.

Let's measure time.

In [7]:
THRESHOLD = 60

def computeSim(limit=None):
  similarity = {}

  lineNodes = sorted(lines.keys())
  nLines = len(lineNodes)

  nComparisons = nLines * (nLines - 1) // 2

  print(f'{nComparisons} comparisons to make')
  chunkSize = nComparisons // 1000

  co = 0
  b = 0
  si = 0
  p = 0

  indent(reset=True)

  stop = False
  for i in range(nLines):
    nodeI = lineNodes[i]
    lineI = lines[nodeI]
    for j in range(i + 1, nLines):
      nodeJ = lineNodes[j]
      lineJ = lines[nodeJ]
      s = sim(lineI, lineJ)
      co += 1
      b += 1
      if b == chunkSize:
        p += 1
        info(f'{p:>3}‰ - {co:>12} comparisons and {si:>10} similarities')
        b = 0
        if limit is not None and p >= limit:
          stop = True
          break

      if s < THRESHOLD:
        continue
      similarity[(nodeI, nodeJ)] = sim(lineI, lineJ)
      si += 1
    if stop:
      break

  info(f'{p:>3}% - {co:>12} comparisons and {si:>10} similarities')
  return similarity

We are going to run it to several ‰ first and do some checks then.

In [12]:
similarity = computeSim(limit=10)

1391782420 comparisons to make
  1.95s   1‰ -      1391782 comparisons and         16 similarities
  3.86s   2‰ -      2783564 comparisons and         28 similarities
  5.77s   3‰ -      4175346 comparisons and         36 similarities
  7.65s   4‰ -      5567128 comparisons and         51 similarities
  9.57s   5‰ -      6958910 comparisons and         71 similarities
    11s   6‰ -      8350692 comparisons and         88 similarities
    13s   7‰ -      9742474 comparisons and        107 similarities
    15s   8‰ -     11134256 comparisons and        125 similarities
    17s   9‰ -     12526038 comparisons and        148 similarities
    19s  10‰ -     13917820 comparisons and        162 similarities
    19s  10% -     13917820 comparisons and        162 similarities


We check the sanity of the results.

In [13]:
print(min(similarity.values()))
print(max(similarity.values()))

60
100


In [14]:
eq = [x for x in similarity.items() if x[1] >= 100]
neq = [x for x in similarity.items() if x[1] <= 70]

In [15]:
print(len(eq))
print(len(neq))

4
62


In [16]:
print(eq[0])
print(neq[0])

((1552976, 1563771), 100)
((1552969, 1563765), 69)


In [17]:
A.plain(eq[0][0][0])
A.plain(eq[0][0][1])

In [18]:
A.plain(eq[0][0][0], fmt='text-trans-full')
A.plain(eq[0][0][1], fmt='text-trans-full')

Looks good.

Now the whole computation.

But if we have done this before, and nothing has changed, we load previous results from disk.

If we do not find previous results, we compute them and save the results to disk.

In [20]:
PARA_DIR = f'{A.tempDir}/parallels'

def writeResults(data, location, name):
  if not os.path.exists(location):
    os.makedirs(location, exist_ok=True)
  path = f'{location}/{name}'
  with gzip.open(path, 'wb') as f:
    pickle.dump(data, f)
  print(f'Data written to {path}')
  
def readResults(location, name):
  path = f'{location}/{name}'
  if not os.path.exists(path):
    print(f'File not found: {path}')
    return None
  with gzip.open(path, 'rb') as f:
    data = pickle.load(f)
  print(f'Data read from {path}')
  return data

In [21]:
similarity = readResults(PARA_DIR, f'sim-{A.version}.zip')
if not similarity:
  similarity = computeSim()
  writeResults(similarity, PARA_DIR, f'sim-{A.version}.zip')

File not found: /Users/dirk/github/etcbc/dss/_temp/parallels/sim-0.3.zip
1391782420 comparisons to make
  1.95s   1‰ -      1391782 comparisons and         16 similarities
  3.88s   2‰ -      2783564 comparisons and         28 similarities
  5.79s   3‰ -      4175346 comparisons and         36 similarities
  7.67s   4‰ -      5567128 comparisons and         51 similarities
  9.59s   5‰ -      6958910 comparisons and         71 similarities
    12s   6‰ -      8350692 comparisons and         88 similarities
    14s   7‰ -      9742474 comparisons and        107 similarities
    16s   8‰ -     11134256 comparisons and        125 similarities
    18s   9‰ -     12526038 comparisons and        148 similarities
    20s  10‰ -     13917820 comparisons and        162 similarities
    22s  11‰ -     15309602 comparisons and        183 similarities
    24s  12‰ -     16701384 comparisons and       4348 similarities
    26s  13‰ -     18093166 comparisons and      10025 similarities
    28s  14‰

In [49]:
len(similarity)

1234

So, just over a thousand pairs of similar lines.

Let's find out which lines have the most correspondences.

In [51]:
parallels = {}

for (l, m) in similarity:
  parallels.setdefault(l, set()).add(m)
  parallels.setdefault(m, set()).add(l)
  
print(f'{len(parallels)} out of {nLines} lines have at least one similar line')

812 out of 6216 ayas have at least one similar aya


In [52]:
rankedParallels = sorted(
  parallels.items(),
  key=lambda x: (-len(x[1]), x[0]),
)

In [53]:
for (l, paras) in rankedParallels[0:10]:
  print(f'{len(paras):>4} siblings of {l} = {T.text(l)} = {T.text(l, fmt="text-source-full", descend=True)}')

  31 siblings of 133058 = فَبِأَىِّ ءَالَآءِ رَبِّكَ تَتَمَارَىٰ = fabi>aY~i 'aAlaA^'i rab~ika tatamaAraY`
  31 siblings of 133133 = فَبِأَىِّ ءَالَآءِ رَبِّكُمَا تُكَذِّبَانِ = fabi>aY~i 'aAlaA^'i rab~ikumaA tuka*~ibaAni
  31 siblings of 133136 = فَبِأَىِّ ءَالَآءِ رَبِّكُمَا تُكَذِّبَانِ = fabi>aY~i 'aAlaA^'i rab~ikumaA tuka*~ibaAni
  31 siblings of 133138 = فَبِأَىِّ ءَالَآءِ رَبِّكُمَا تُكَذِّبَانِ = fabi>aY~i 'aAlaA^'i rab~ikumaA tuka*~ibaAni
  31 siblings of 133141 = فَبِأَىِّ ءَالَآءِ رَبِّكُمَا تُكَذِّبَانِ = fabi>aY~i 'aAlaA^'i rab~ikumaA tuka*~ibaAni
  31 siblings of 133143 = فَبِأَىِّ ءَالَآءِ رَبِّكُمَا تُكَذِّبَانِ = fabi>aY~i 'aAlaA^'i rab~ikumaA tuka*~ibaAni
  31 siblings of 133145 = فَبِأَىِّ ءَالَآءِ رَبِّكُمَا تُكَذِّبَانِ = fabi>aY~i 'aAlaA^'i rab~ikumaA tuka*~ibaAni
  31 siblings of 133148 = فَبِأَىِّ ءَالَآءِ رَبِّكُمَا تُكَذِّبَانِ = fabi>aY~i 'aAlaA^'i rab~ikumaA tuka*~ibaAni
  31 siblings of 133150 = فَبِأَىِّ ءَالَآءِ رَبِّكُمَا تُكَذِّبَانِ = fabi>aY~i 'aAlaA^

In [54]:
for (l, paras) in rankedParallels[100:110]:
  print(f'{len(paras):>4} siblings of {T.text(l)} = {T.text(l, fmt="text-source-full", descend=True)}')

   4 siblings of إِنِّى لَكُمْ رَسُولٌ أَمِينٌ = <in~iY lakumo rasuwlN >amiynN
   4 siblings of إِنِّى لَكُمْ رَسُولٌ أَمِينٌ = <in~iY lakumo rasuwlN >amiynN
   4 siblings of إِنِّى لَكُمْ رَسُولٌ أَمِينٌ = <in~iY lakumo rasuwlN >amiynN
   4 siblings of فِى جَنَّٰتٍ وَعُيُونٍ = fiY jan~a`tK waEuyuwnK
   4 siblings of إِنِّى لَكُمْ رَسُولٌ أَمِينٌ = <in~iY lakumo rasuwlN >amiynN
   4 siblings of إِنِّى لَكُمْ رَسُولٌ أَمِينٌ = <in~iY lakumo rasuwlN >amiynN
   4 siblings of طسٓ تِلْكَ ءَايَٰتُ ٱلْقُرْءَانِ وَكِتَابٍ مُّبِينٍ = Ts^ tiloka 'aAya`tu {loquro'aAni wakitaAbK m~ubiynK
   4 siblings of تِلْكَ ءَايَٰتُ ٱلْكِتَٰبِ ٱلْحَكِيمِ = tiloka 'aAya`tu {lokita`bi {loHakiymi
   4 siblings of إِنَّا كَذَٰلِكَ نَجْزِى ٱلْمُحْسِنِينَ = <in~aA ka*a`lika najoziY {lomuHosiniyna
   4 siblings of كَذَٰلِكَ نَجْزِى ٱلْمُحْسِنِينَ = ka*a`lika najoziY {lomuHosiniyna


In [55]:
for (l, paras) in rankedParallels[500:510]:
  print(f'{len(paras):>4} siblings of {T.text(l)} = {T.text(l, fmt="text-source-full", descend=True)}')

   1 siblings of يَأْتُوكَ بِكُلِّ سَحَّارٍ عَلِيمٍ = ya>otuwka bikul~i saH~aArK EaliymK
   1 siblings of فَلَمَّا جَآءَ ٱلسَّحَرَةُ قَالُوا۟ لِفِرْعَوْنَ أَئِنَّ لَنَا لَأَجْرًا إِن كُنَّا نَحْنُ ٱلْغَٰلِبِينَ = falam~aA jaA^'a {ls~aHarapu qaAluwA@ lifiroEawona >a}in~a lanaA la>ajorFA <in kun~aA naHonu {loga`libiyna
   1 siblings of قَالَ نَعَمْ وَإِنَّكُمْ إِذًا لَّمِنَ ٱلْمُقَرَّبِينَ = qaAla naEamo wa<in~akumo <i*FA l~amina {lomuqar~abiyna
   1 siblings of فَأُلْقِىَ ٱلسَّحَرَةُ سَٰجِدِينَ = fa>uloqiYa {ls~aHarapu sa`jidiyna
   1 siblings of قَالُوٓا۟ ءَامَنَّا بِرَبِّ ٱلْعَٰلَمِينَ = qaAluw^A@ 'aAman~aA birab~i {loEa`lamiyna
   1 siblings of رَبِّ مُوسَىٰ وَهَٰرُونَ = rab~i muwsaY` waha`ruwna
   1 siblings of قَالَ ءَامَنتُمْ لَهُۥ قَبْلَ أَنْ ءَاذَنَ لَكُمْ إِنَّهُۥ لَكَبِيرُكُمُ ٱلَّذِى عَلَّمَكُمُ ٱلسِّحْرَ فَلَسَوْفَ تَعْلَمُونَ لَأُقَطِّعَنَّ أَيْدِيَكُمْ وَأَرْجُلَكُم مِّنْ خِلَٰفٍ وَلَأُصَلِّبَنَّكُمْ أَجْمَعِينَ = qaAla 'aAmantumo lahu, qabola >ano 'aA*ana lakumo <in~ahu, 

And how many lines have just one correspondence?

We look at the tail of rankedParallels.

In [56]:
pairs = [(x, list(paras)[0]) for (x, paras) in rankedParallels if len(paras) == 1]
print(f'There are {len(pairs)} exclusively parallel pairs of lines')

There are 536 exclusively parallel pairs of lines


Why not make an overview of exactly how wide-spread parallel lines are?

We count how many lines have how many parallels.

In [57]:
parallelCount = collections.Counter()

buckets = (2, 10, 20, 50, 100)

bucketRep = {}
prevBucket = None
for bucket in buckets:
  if prevBucket is None:
    bucketRep[bucket] = f'       n <= {bucket:>3}'
  elif bucket == buckets[-1]:
    bucketRep[bucket] = f'       n >  {bucket:>3}'
  else:
    bucketRep[bucket] = f'{prevBucket:>3} <  n <= {bucket:>3}'
  prevBucket = bucket

for (l, paras) in rankedParallels:
  clusterSize = len(paras) + 1
  if clusterSize > buckets[-1]:
    theBucket = buckets[-1]
  else:
    for bucket in buckets:
      if clusterSize <= bucket:
        theBucket = bucket
        break
  parallelCount[theBucket] += 1
  
for (bucket, amount) in sorted(
  parallelCount.items(),
  key=lambda x: (-x[0], x[1]),
):
  print(f'{amount:>4} lines have n sisters where {bucketRep[bucket]}')

  32 lines have n sisters where  20 <  n <=  50
  12 lines have n sisters where  10 <  n <=  20
 232 lines have n sisters where   2 <  n <=  10
 536 lines have n sisters where        n <=   2


# Add parallels to the TF dataset

We can add this information to the DSS dataset as an *edge feature*.

An edge feature links two nodes and may annotate that link with a value.

For parallels, we link each line to each of its parallel lines and we annotate that link with the similarity between
the two lines. The similarity is a percentage, and we round it to integer values.

If *n1* is similar to *n2*, then *n2* is similar to *n1*.
In order to save space, we only add such links once.

We can then use
[`E.sim.b(node)`](https://annotation.github.io/text-fabric/Api/Features/#edge-features)
to find all nodes that are parallel to node.


In [22]:
metaData = {
  '': {
    'acronym': 'dss',
    'description': 'parallel lines in the DSS (computed)'
    'createdBy: 'Dirk Roorda',
    'createdDate: '2019-05-09',
    'sourceCreatedDate: '2015',
    'sourceCreatedBy: 'Martin G. Abegg, Jr., James E. Bowley, and Edward M. Cook',
    'convertedBy: 'Jarod Jacobs, Martijn Naaijer and Dirk Roorda',
    'source: "Martin Abegg's data files, personal communication",
    'license: 'Creative Commons Attribution-NonCommercial 4.0 International License',
    'licenseUrl: 'http://creativecommons.org/licenses/by-nc/4.0/',
    'sourceDescription: 'Dead Sea Scrolls: biblical and non-biblical scrolls',
  },
  'sim': {
    'valueType': 'int',
    'edgeValues': True,
    'description': 'similarity between lines, as a percentage of the common material wrt the combined material',
  },
}

SyntaxError: invalid syntax (<ipython-input-22-d3e8fd32c8f6>, line 5)

In [59]:
simData = {}
for ((f, t), d) in similarity.items():
  simData.setdefault(f, {})[t] = d

In [60]:
ghBase = os.path.expanduser('~/github')
subdir = 'parallels'
path = f'{A.org}/{A.repo}/{subdir}/tf'
location = f'{ghBase}/{path}'
module = A.version

In [61]:
TF.save(edgeFeatures=dict(sim=simData), metaData=metaData, location=location, module=module)

   |     0.00s T sim                  to /Users/dirk/github/q-ran/quran/parallels/tf/0.3


True

# Turn the parallels feature into a module

Here we show how to turn the new feature `sim` into a module, so that users can easily load it in a Jupyter notebook or in the TF browser.

In [62]:
%%bash
text-fabric-zip 'etcbc/dss/parallels/tf'

True
Create release data for q-ran/quran/parallels/tf
zip files end up in /Users/dirk/Downloads/q-ran-release/quran
zipping q-ran/quran                0.3 with   1 features ==> parallels-tf-0.3.zip


I have added this file to a new release of the DSS Github repo.

# Use the parallels module

We load the DSS corpus again, but now with the parallels module.

In [63]:
A = use('dss:clone', checkout='clone', hoist=globals(), mod='etcbc/dss/parallels/tf')

TF app is up-to-date.
Using annotation/app-quran commit f6543c213dad36050de3e90373af237e9a4f0bc1 (=latest)
  in /Users/dirk/text-fabric-data/__apps__/quran.
No new data release available online.
Using q-ran/quran/tf - 0.3 rv0.4 (=latest) in /Users/dirk/text-fabric-data.
	downloading q-ran/quran - 0.3 rv0.4
	from https://github.com/q-ran/quran/releases/download/v0.4/parallels-tf-0.3.zip ...
	unzipping ...
	saving q-ran/quran - 0.3 rv0.4
	saved q-ran/quran - 0.3 rv0.4
Using q-ran/quran/parallels/tf - 0.3 rv0.4 (=latest) in /Users/dirk/text-fabric-data


Lo and behold: you see the parallels module listed with one feature: `sim`. It is in *italics*, which indicates
it is an edge feature.

We just do a quick check here and in another notebook we study parallels a bit more, using the feature `sim`.

We count how many similar pairs their are, and how many 100% similar pairs there are.

In [64]:
query = '''
line
-sim> line
'''
results = A.search(query)

  0.02s 1234 results


In [65]:
query = '''
line
-sim=100> line
'''
results = A.search(query)

  0.02s 805 results


Remarkably, more than half of the pairs are 100 percent similar. Let's show just a few:

In [66]:
A.table(results, start=1, end=10, withNodes=True)

n,p,aya,aya
1,37:182,ٱلْحَمْدُ لِلَّهِ رَبِّ ٱلْعَٰلَمِينَ 128221,وَٱلْحَمْدُ لِلَّهِ رَبِّ ٱلْعَٰلَمِينَ 132189
2,4:68,ٱهْدِنَا ٱلصِّرَٰطَ ٱلْمُسْتَقِيمَ 128225,وَلَهَدَيْنَٰهُمْ صِرَٰطًا مُّسْتَقِيمًا 128780
3,37:118,ٱهْدِنَا ٱلصِّرَٰطَ ٱلْمُسْتَقِيمَ 128225,وَهَدَيْنَٰهُمَا ٱلصِّرَٰطَ ٱلْمُسْتَقِيمَ 132125
4,31:5,أُو۟لَٰٓئِكَ عَلَىٰ هُدًى مِّن رَّبِّهِمْ وَأُو۟لَٰٓئِكَ هُمُ ٱلْمُفْلِحُونَ 128231,أُو۟لَٰٓئِكَ عَلَىٰ هُدًى مِّن رَّبِّهِمْ وَأُو۟لَٰٓئِكَ هُمُ ٱلْمُفْلِحُونَ 131693
5,2:122,يَٰبَنِىٓ إِسْرَٰٓءِيلَ ٱذْكُرُوا۟ نِعْمَتِىَ ٱلَّتِىٓ أَنْعَمْتُ عَلَيْكُمْ وَأَنِّى فَضَّلْتُكُمْ عَلَى ٱلْعَٰلَمِينَ 128273,يَٰبَنِىٓ إِسْرَٰٓءِيلَ ٱذْكُرُوا۟ نِعْمَتِىَ ٱلَّتِىٓ أَنْعَمْتُ عَلَيْكُمْ وَأَنِّى فَضَّلْتُكُمْ عَلَى ٱلْعَٰلَمِينَ 128348
6,2:141,تِلْكَ أُمَّةٌ قَدْ خَلَتْ لَهَا مَا كَسَبَتْ وَلَكُم مَّا كَسَبْتُمْ وَلَا تُسْـَٔلُونَ عَمَّا كَانُوا۟ يَعْمَلُونَ 128360,تِلْكَ أُمَّةٌ قَدْ خَلَتْ لَهَا مَا كَسَبَتْ وَلَكُم مَّا كَسَبْتُمْ وَلَا تُسْـَٔلُونَ عَمَّا كَانُوا۟ يَعْمَلُونَ 128367
7,3:60,ٱلْحَقُّ مِن رَّبِّكَ فَلَا تَكُونَنَّ مِنَ ٱلْمُمْتَرِينَ 128373,ٱلْحَقُّ مِن رَّبِّكَ فَلَا تَكُن مِّنَ ٱلْمُمْتَرِينَ 128572
8,3:88,خَٰلِدِينَ فِيهَا لَا يُخَفَّفُ عَنْهُمُ ٱلْعَذَابُ وَلَا هُمْ يُنظَرُونَ 128388,خَٰلِدِينَ فِيهَا لَا يُخَفَّفُ عَنْهُمُ ٱلْعَذَابُ وَلَا هُمْ يُنظَرُونَ 128600
9,19:36,إِنَّ ٱللَّهَ رَبِّى وَرَبُّكُمْ فَٱعْبُدُوهُ هَٰذَا صِرَٰطٌ مُّسْتَقِيمٌ 128563,وَإِنَّ ٱللَّهَ رَبِّى وَرَبُّكُمْ فَٱعْبُدُوهُ هَٰذَا صِرَٰطٌ مُّسْتَقِيمٌ 130505
10,43:64,إِنَّ ٱللَّهَ رَبِّى وَرَبُّكُمْ فَٱعْبُدُوهُ هَٰذَا صِرَٰطٌ مُّسْتَقِيمٌ 128563,إِنَّ ٱللَّهَ هُوَ رَبِّى وَرَبُّكُمْ فَٱعْبُدُوهُ هَٰذَا صِرَٰطٌ مُّسْتَقِيمٌ 132608


There is also a lower level way to work with edge features.

We can list all edges going out from a reference node.
What we see is tuple of pairs: the target node and the similarity between the reference node and that target node.

In [70]:
refNode = 133152

E.sim.f(refNode)

((133154, 100),
 (133156, 100),
 (133158, 100),
 (133160, 100),
 (133162, 100),
 (133165, 100),
 (133167, 100),
 (133169, 100),
 (133171, 100),
 (133173, 100),
 (133175, 100),
 (133177, 100),
 (133179, 100),
 (133181, 100),
 (133183, 100),
 (133185, 100),
 (133187, 100),
 (133189, 100),
 (133191, 100),
 (133193, 100),
 (133195, 100),
 (133197, 100))

Likewise, we can observe the nodes that target the reference node:

In [71]:
E.sim.t(refNode)

((133058, 60),
 (133133, 100),
 (133136, 100),
 (133138, 100),
 (133141, 100),
 (133143, 100),
 (133145, 100),
 (133148, 100),
 (133150, 100))

Both sets of nodes are similar to the reference node and it is inconvenient to use both `.f()` and `.t()` to get the similar lines.

But there is another way:

In [72]:
E.sim.b(refNode)

((133058, 60),
 (133133, 100),
 (133136, 100),
 (133138, 100),
 (133141, 100),
 (133143, 100),
 (133145, 100),
 (133148, 100),
 (133150, 100),
 (133154, 100),
 (133156, 100),
 (133158, 100),
 (133160, 100),
 (133162, 100),
 (133165, 100),
 (133167, 100),
 (133169, 100),
 (133171, 100),
 (133173, 100),
 (133175, 100),
 (133177, 100),
 (133179, 100),
 (133181, 100),
 (133183, 100),
 (133185, 100),
 (133187, 100),
 (133189, 100),
 (133191, 100),
 (133193, 100),
 (133195, 100),
 (133197, 100))

Let's make sure that `.b()` gives the combination of `.f()` and `.t()`.

In [73]:
f = {x[0] for x in E.sim.f(refNode)}
b = {x[0] for x in E.sim.b(refNode)}
t = {x[0] for x in E.sim.t(refNode)}

# are f and t disjoint ?

print(f'the intersection of f and t is {f & t}')

# is b the union of f and t ?

print(f't | f = b ? {f | t == b}')

the intersection of f and t is set()
t | f = b ? True
